In [14]:
import io 
import sagemaker
import boto3
import json
from sagemaker import get_execution_role

import base64


In [15]:

role = get_execution_role()
region = boto3.Session().region_name
print(role, region)

sagemaker_session = sagemaker.session.Session()
sm_client = boto3.client("sagemaker", region_name=region)
sm_runtime_client = boto3.client("sagemaker-runtime")




arn:aws:iam::396608815783:role/service-role/AmazonSageMaker-ExecutionRole-20250207T102425 us-west-2


In [16]:
# SageMaker Endpoint 이름
# endpoint_name = sagemaker.utils.name_from_base("QWEN-2-5-7B-G5")
# endpoint_name = "QWEN-2-5-7B-G5-2025-08-12-03-05-19-368"


all_endpoint = sm_client.list_endpoints()
print(all_endpoint,"\n")
endpoint_name = all_endpoint['Endpoints'][0]['EndpointName']
print(" - endpoint_name : ", endpoint_name)

image_path = "images/test001.png"

{'Endpoints': [{'EndpointName': 'gemma3-s3-vllm-async-2025-08-14-02-10-03-005', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:396608815783:endpoint/gemma3-s3-vllm-async-2025-08-14-02-10-03-005', 'CreationTime': datetime.datetime(2025, 8, 14, 2, 10, 5, 750000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2025, 8, 14, 2, 19, 17, 309000, tzinfo=tzlocal()), 'EndpointStatus': 'InService'}, {'EndpointName': 'gemma3-s3-vllm-async-2025-08-14-01-16-07-849', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:396608815783:endpoint/gemma3-s3-vllm-async-2025-08-14-01-16-07-849', 'CreationTime': datetime.datetime(2025, 8, 14, 1, 16, 9, 400000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2025, 8, 14, 1, 25, 31, 472000, tzinfo=tzlocal()), 'EndpointStatus': 'InService'}, {'EndpointName': 'QWEN-2-5-7B-G5-2025-08-12-03-05-19-368', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:396608815783:endpoint/QWEN-2-5-7B-G5-2025-08-12-03-05-19-368', 'CreationTime': datetime.datetime(2025, 8, 12, 3, 5, 

In [17]:
predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer(),
)


In [33]:
# 1. 이미지 파일을 base64로 인코딩
with open(image_path, "rb") as f:
    image_b64 = base64.b64encode(f.read()).decode("utf-8")


payload = {
    "messages": [
        {
            "role": "user",
            "content": [
                # {"type": "image", "data": image_b64},
                # {"type": "text", "text": "이 이미지는 무엇인가요?"}
                # {"type": "text", "text": " Ai Agent 무엇인가요?"}
                # {"type": "text", "text": "이 이미지는 무엇인가요?"}
                {"type": "text", "text": "안녕하세요."},
                # {"type": "image", "image_url": "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG"},
                # {"type": "image", "data": image_b64},
                
                
            ]
        }
    ],
    "max_tokens": 256,
    "temperature": 0.8
}

payload = {
    "messages": [
        {
            "role": "user",
            "content": [
                # {"type": "text", "text": "hello."},
                
                {"type": "image", "image_base64": image_b64},
                {"type": "text", "text": "이 이미지의 내용을 설명해 주세요."},
            ]
        }
    ]
}

single_image_payload = {
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG",
                    }
                },
                {
                    "type": "text",
                    "text": "이미지에 대해서 설명해줘.",
                },
                
            ]
        }
    ],
    "max_tokens": 1024,
    "temperature": 0.6,
    "top_p": 0.9,
}


In [34]:
# JSON 문자열로 변환
# payload_json = json.dumps(payload)

# 엔드포인트에 요청 보내기
# response = predictor.predict(data=payload_json)
response = predictor.predict(single_image_payload)

In [35]:
response

{'id': 'chatcmpl-e5f2c55234a74bfe83458d73ae24f745',
 'object': 'chat.completion',
 'created': 1755139169,
 'model': 'lmi',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'reasoning_content': None,
    'content': '이미지에 대한 설명입니다.\n\n**전반적인 내용:**\n\n*   **주요 피사체:** 한 손에 여러 개의 캔디가 들어 있는 모습입니다.\n*   **배경:** 밝은 색상의 나무 바닥이 배경으로 보입니다.\n*   **조명:** 자연광 또는 밝은 조명 아래 촬영되었으며, 손과 캔디에 그림자가 드리워져 있습니다.\n\n**캔디에 대한 설명:**\n\n*   **색상:** 캔디는 다양한 색상으로, 파란색, 주황색, 초록색, 보라색 등으로 보입니다.\n*   **디자인:** 각 캔디에는 독특한 디자인이 새겨져 있습니다.\n    *   파란색 캔디에는 잎 모양이 그려져 있습니다.\n    *   보라색 캔디에는 작은 동물 모양이 그려져 있습니다.\n    *   초록색 캔디에는 잎 모양이 그려져 있습니다.\n    *   주황색 캔디에는 잎 모양이 그려져 있습니다.\n*   **모양:** 캔디는 타원형 또는 알갱이 모양입니다.\n\n**손에 대한 설명:**\n\n*   **피부:** 밝은 피부톤을 가진 손입니다.\n*   **옷:** 손가락 일부가 보이는 옷은 갈색의 니트 또는 스웨터 같습니다.\n\n**전반적인 분위기:**\n\n*   **느낌:** 캔디를 들고 있는 모습에서 즐거움과 흥미를 느끼게 합니다.\n*   **스타일:** 평범하고 일상적인 순간을 포착한 사진입니다.\n\n이 설명이 이미지에 대한 이해에 도움이 되었기를 바랍니다.',
    'tool_calls': []},
   'logprobs': None,
   'finish_reason':

In [36]:
response['choices'][0]['message']

{'role': 'assistant',
 'reasoning_content': None,
 'content': '이미지에 대한 설명입니다.\n\n**전반적인 내용:**\n\n*   **주요 피사체:** 한 손에 여러 개의 캔디가 들어 있는 모습입니다.\n*   **배경:** 밝은 색상의 나무 바닥이 배경으로 보입니다.\n*   **조명:** 자연광 또는 밝은 조명 아래 촬영되었으며, 손과 캔디에 그림자가 드리워져 있습니다.\n\n**캔디에 대한 설명:**\n\n*   **색상:** 캔디는 다양한 색상으로, 파란색, 주황색, 초록색, 보라색 등으로 보입니다.\n*   **디자인:** 각 캔디에는 독특한 디자인이 새겨져 있습니다.\n    *   파란색 캔디에는 잎 모양이 그려져 있습니다.\n    *   보라색 캔디에는 작은 동물 모양이 그려져 있습니다.\n    *   초록색 캔디에는 잎 모양이 그려져 있습니다.\n    *   주황색 캔디에는 잎 모양이 그려져 있습니다.\n*   **모양:** 캔디는 타원형 또는 알갱이 모양입니다.\n\n**손에 대한 설명:**\n\n*   **피부:** 밝은 피부톤을 가진 손입니다.\n*   **옷:** 손가락 일부가 보이는 옷은 갈색의 니트 또는 스웨터 같습니다.\n\n**전반적인 분위기:**\n\n*   **느낌:** 캔디를 들고 있는 모습에서 즐거움과 흥미를 느끼게 합니다.\n*   **스타일:** 평범하고 일상적인 순간을 포착한 사진입니다.\n\n이 설명이 이미지에 대한 이해에 도움이 되었기를 바랍니다.',
 'tool_calls': []}